# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [10]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [11]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key
)

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [12]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [13]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [14]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [15]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [16]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [17]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'skills page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'project page - Connect Four',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page - Outsmart',
   'url': 'https://edwarddonner.com/outsmart/'}]}

In [18]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [19]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [20]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'models hub', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets hub', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces hub', 'url': 'https://huggingface.co/spaces'},
  {'type': 'inference endpoints', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discussions foru

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [21]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [22]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-397B-A17B
Updated
about 4 hours ago
•
303k
•
923
nvidia/personaplex-7b-v1
Updated
8 days ago
•
539k
•
2.16k
Nanbeige/Nanbeige4.1-3B
Updated
1 day ago
•
178k
•
739
zai-org/GLM-5
Updated
10 days ago
•
179k
•
1.46k
MiniMaxAI/MiniMax-M2.5
Updated
7 days ago
•
209k
•
868
Browse 2M+ models
Spaces
Running
on
Zero
MCP
887
Wan2.2 14B Preview
🐌
887
generate a video from an image with a text prompt
Running
on
Zer

In [23]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [25]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-397B-A17B\nUpdated\nabout 4 hours ago\n•\n303k\n•\n923\nnvidia/personaplex-7b-v1\nUpdated\n8 days ago\n•\n539k\n•\n2.16k\nNanbeige/Nanbeige4.1-3B\nUpdated\n1 day ago\n•\n178k\n•\n739\nzai-org/GLM-5\nUpdated\n10 days ago\n•\n179k\n•\n1.46k\nMiniMaxAI/MiniM

In [26]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [27]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


# Hugging Face Brochure

---

## About Hugging Face
Hugging Face is a vibrant collaboration platform dedicated to the global machine learning (ML) community. It serves as the central hub for sharing, exploring, discovering, and experimenting with open-source machine learning models, datasets, and applications. Hugging Face empowers ML engineers, scientists, and users to build an open and ethical AI future together.

---

## Our Mission
We are building the **future of AI** by fostering a robust community where machine learning enthusiasts and professionals collaborate freely. Our goal is to make ML more accessible, accelerate innovation, and nurture ethical AI development for everyone.

---

## What We Offer

- **Hugging Face Hub**: Explore over 2 million models, 500,000+ datasets, and 1 million+ applications across modalities such as text, image, video, audio, and even 3D.  
- **Spaces**: Run and share AI-powered apps easily with a supportive community backing.  
- **Open Source Stack**: Move faster in your ML projects with a comprehensive open-source technology suite.  
- **Inference Made Easy**: Start running AI inference within seconds using accessible cloud compute resources.  
- **Community & Collaboration**: Host unlimited public models and datasets and collaborate with peers worldwide.

---

## Popular Resources

- **Trending Models** like Qwen3.5-397B-A17B and personaplex-7b-v1.  
- **Spaces Demos** such as image to video generation, custom speech synthesis, and image editing tools.  
- **Datasets** covering reasoning, OCR, mapping, and coding benchmarks.

---

## Company Culture
Hugging Face thrives on openness, creativity, and collaboration. Our culture celebrates the free exchange of ideas to push the boundaries of AI and machine learning. We are united by a commitment to ethical AI, community-driven innovation, and empowering the next generation of AI practitioners.

---

## Customers & Community
Our diverse user base includes ML researchers, enterprises, developers, and AI enthusiasts globally. We support everyone from beginners building a portfolio to advanced researchers deploying cutting-edge models. Many enterprise clients leverage our platform for scalable AI solutions, backed by our open-source leadership.

---

## Careers & Opportunities
Join Hugging Face and contribute to shaping the future of AI! We offer roles across engineering, research, community building, and enterprise services. Be part of a fast-growing company that values innovation, diversity, and a collaborative spirit. Check out available job openings on our website and become a key player in the AI revolution.

---

## How to Get Involved

- **Sign Up** for free to start exploring models and datasets.  
- **Build Your Portfolio** by sharing your machine learning projects.  
- **Participate in the Community** by contributing to open-source projects and forums.  
- **Explore Enterprise Solutions** for scalable AI adoption in your business.

---

## Contact & More Information

Visit our official site: [huggingface.co](https://huggingface.co)  
Explore models, datasets, and applications or connect with our vibrant AI community.

---

Hugging Face — The AI community building the future.  
Together, we create, innovate, and shape the open and ethical future of machine learning.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [28]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future. It is the premier collaboration platform for the machine learning (ML) community, designed to enable improved creation, discovery, and sharing of models, datasets, and applications across all AI modalities including text, images, video, audio, and even 3D.

The platform serves as a central hub where ML engineers, scientists, and end users come together to collaborate on innovative open-source machine learning projects—driving forward an open, ethical AI future.

---

## What Hugging Face Offers

- **2M+ Models**: Access and contribute to a vast and constantly expanding library of machine learning models covering a wide range of use cases and technologies. Popular trending models include Qwen3.5, Nvidia’s Personaplex, Nanbeige, GLM-5, MiniMaxAI, and many more.

- **500K+ Datasets**: Discover extensive datasets from diverse domains that power cutting-edge AI research and applications.

- **1M+ Applications (Spaces)**: Engage with interactive AI-powered applications like text-to-speech demos, image generation and editing tools, video generation from images, and animation tools, all hosted and run on Hugging Face Spaces.

- **Open Source Stack**: Utilize Hugging Face's open-source software, accelerating your development cycle and integration capabilities.

- **Paid Compute**: Leverage scalable paid compute resources for faster training and inference, making AI development more efficient.

---

## Community and Collaboration

- Hugging Face is more than just a platform—it's a rapidly growing community and ecosystem where machine learning practitioners collaborate globally.
- Users can host unlimited public projects, explore others’ work, and build their own AI portfolios.
- The platform fosters transparency and ethical AI development.

---

## Company Culture

- **Open and Ethical AI**: Hugging Face champions transparency and openness in AI research and development.
- **Inclusive Collaboration**: Encourages an active global community of innovators to share knowledge and build together.
- **Empowering Developers**: Provides tools and resources that enable the next generation of ML engineers and scientists to shine.
- **Innovation-Driven**: Continuously evolving with the latest models and technologies such as GGML and llama.cpp integration.

---

## Customers & Users

- A diverse set of users from independent ML researchers, developers, startups, large enterprises, and academic institutions.
- Customers rely on Hugging Face for a comprehensive ML ecosystem to accelerate AI projects, from experimentation to deployment.
- Enterprises benefit from the Hugging Face Enterprise offering combined with scalable compute solutions.

---

## Careers at Hugging Face

- Join a pioneering AI company dedicated to building an open and collaborative AI future.
- Opportunities available for passionate machine learning engineers, data scientists, researchers, software developers, and community builders.
- Work in a vibrant tech culture driven by innovation, inclusivity, and impact.
- Access to cutting-edge AI tools, large-scale collaborative projects, and a passionate global community.

---

## Contact & Get Started

- **Website**: https://huggingface.co
- **Sign Up**: Join the community to share, explore and accelerate your machine learning journey.
- **Try**: Chat with AI using HuggingChat Omni or browse models, datasets, and spaces instantly.
- Build your AI portfolio while contributing to the future of machine learning.

---

## Brand Assets

- Recognized by the signature yellow (#FFD21E) and orange (#FF9D00) color scheme.
- The logo and brand assets symbolize openness and technological progress.

---

Hugging Face is your gateway to the world of collaborative machine learning — join the AI revolution today!

In [31]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning. It is a collaborative platform designed for machine learning engineers, scientists, and enthusiasts to share, explore, discover, and experiment with open-source models, datasets, and applications.

At its core, Hugging Face empowers the next generation of machine learning professionals to collaborate and innovate, fostering an open and ethical AI future.

---

## What We Offer

- **Models:** Access and contribute to over **2 million+ machine learning models** spanning text, image, video, audio, and even 3D modalities. Models are continuously updated, ensuring freshness and relevance.

- **Datasets:** Explore and contribute to a rich repository of **500,000+ datasets** covering various domains to accelerate your machine learning projects.

- **Spaces:** Host and discover machine learning applications and demos, with over a million applications available. Spaces support running models and apps, making it easy to visualize and test AI capabilities.

- **Collaboration Platform:** Host unlimited public models, datasets, and applications to enable faster innovation and knowledge sharing.

- **Compute Services:** Hugging Face offers paid compute resources to accelerate machine learning model inference and deployment, helping users to scale with ease.

---

## Community & Culture

Hugging Face fosters a **vibrant and fast-growing community** centered on openness and ethical AI development. The platform serves as a central place for:

- Sharing work openly in support of scientific progress and responsible AI use.
- Building professional portfolios by publishing models and projects.
- Collaborating worldwide with ML engineers, researchers, and developers.

The company culture emphasizes **collaboration, transparency, and innovation**, inviting contributors to be part of shaping AI’s future responsibly.

---

## Customers & Use Cases

Hugging Face serves a diverse range of users including:

- **AI Researchers and Scientists** looking to publish and share open-source work.
- **Machine Learning Engineers** utilizing pre-trained models and datasets to build applications faster.
- **Enterprises** seeking customized AI solutions, with access to enterprise-grade services.
- **Developers and Creators** building AI demos and leveraging AI-powered tools in apps.

Trending models on the platform include state-of-the-art language models, speech synthesis tools, image generation, and video animation, powering cutting-edge applications.

---

## Careers at Hugging Face

Join a company at the forefront of AI innovation:

- Work alongside top AI experts in a collaborative and inclusive environment.
- Contribute to open-source projects and drive the future of ethical AI.
- Engage with a passionate community globally.
- Opportunities range from engineering, research, developer relations, community management, and more.

Stay tuned to the Hugging Face website for current job openings.

---

## Get Started

- Explore AI apps and models at the Hugging Face Hub.
- Sign up for free to start your journey of discovery, experimentation, and collaboration.
- Access powerful compute resources to accelerate your AI projects.

Visit: [huggingface.co](https://huggingface.co)

---

## Brand Identity

The brand is marked by the signature vibrant yellow (#FFD21E) and orange (#FF9D00) colors, symbolizing innovation and energy. Hugging Face logos are available in various formats to represent the community-driven spirit of the company.

---

**Hugging Face** — Empowering the Machine Learning Community to build an open, ethical, and innovative AI future.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>